환경설정

In [29]:
import os
import pandas as pd
import numpy as np
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
import time
import tqdm
import jpype
import ast

In [30]:
# 전처리, 명사추출, 키워드추출, 불용어후처리까지 마친 데이터
folder_path = '/Users/jaesolshin/Documents/GitHub/youtube_dashboard'
file_path = os.path.join(folder_path, 'keyword_list_after_cleaning.csv')
comments_df = pd.read_csv(file_path)

# csv 파일로 저장되면서 문자열로 변형된 word_list 컬럼을 다시 리스트로 변환
sample_df = comments_df.copy()
sample_df['word_list'] = sample_df['word_list'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# 키워드 추출

In [31]:
import time
from collections import defaultdict

def extract_keywords_frequency(series_word_list, top_n=30, min_df=1, existing_keywords=None):
    # 시작시간 확인
    start_time = time.time()

    # 기존 키워드가 없으면 빈 리스트로 초기화
    if existing_keywords is None:
        existing_keywords = []

    # 단어별 문서 빈도를 저장할 딕셔너리 (각 단어가 등장한 문서 수)
    doc_frequency = defaultdict(int)

    # 각 문서에서 등장한 단어 추출 및 빈도 계산
    for word_list in series_word_list:
        unique_words_in_doc = list(set(word_list))  # 문서 내에서 중복된 단어는 한 번만 카운트
        for word in unique_words_in_doc:
            doc_frequency[word] += 1

    # min_df 조건을 충족하는 단어들의 빈도를 저장할 딕셔너리
    word_frequencies = defaultdict(int)

    # 각 문서의 단어 빈도 카운트 (min_df 조건에 맞는 단어만 빈도 계산)
    for word_list in series_word_list:
        for word in word_list:
            if doc_frequency[word] >= min_df:
                word_frequencies[word] += 1

    # 상위 빈도 단어 추출
    sorted_words = sorted(word_frequencies.items(), key=lambda item: item[1], reverse=True)

    # 기존 키워드 제외하고 새로운 키워드만 추출
    new_keywords_with_weights = [(word, count) for word, count in sorted_words if word not in existing_keywords]
    
    # 상위 n개 키워드만 추출 (중복 제외 후)
    top_keywords_with_weights = new_keywords_with_weights[:top_n]

    # 키워드만 추출
    top_keywords = [word for word, _ in top_keywords_with_weights]

    # 가중치를 전체 합으로 표준화
    total_count = sum(count for _, count in top_keywords_with_weights)
    if total_count > 0:
        keywords_with_weights = [(word, count / total_count) for word, count in top_keywords_with_weights]
    else:
        keywords_with_weights = [(word, 0) for word, count in top_keywords_with_weights]

    # 종료시간 확인
    end_time = time.time()

    # 소요시간 출력
    print(f"{len(series_word_list)}개 댓글 키워드 추출에 걸린 시간: {end_time - start_time} 초\n")

    return top_keywords, keywords_with_weights


In [32]:
# TF-IDF 기반 키워드 및 상대적 비중 추출 함수
def extract_keywords_tfidf(series_word_list, top_n=100, min_df=1):

    # 시작시간 확인
    start_time = time.time()

    # 리스트 시리즈를 문자열 시리즈로 변형
    word_list_str = series_word_list.apply(lambda words: ' '.join(words))

    # TF-IDF 벡터화 (min_df를 설정해 빈도수가 적은 단어 제거)
    vectorizer = TfidfVectorizer(max_features=1000, min_df=min_df)
    tfidf_matrix = vectorizer.fit_transform(word_list_str)

    # 각 단어의 TF-IDF 점수를 추출
    feature_names = vectorizer.get_feature_names_out()
    tfidf_scores = np.mean(tfidf_matrix.toarray(), axis=0)  # 전체 문서에서 평균 TF-IDF 추출

    # TF-IDF 점수를 기준으로 상위 키워드 추출
    top_n_indices = tfidf_scores.argsort()[-top_n:][::-1]
    top_keywords = [feature_names[idx] for idx in top_n_indices]

    # 각 단어의 TF-IDF 점수 추출
    weights = [tfidf_scores[idx] for idx in top_n_indices]

    # 가중치를 전체 합으로 표준화
    weights = weights / np.sum(weights)

    # 키워드와 가중치를 묶어서 반환
    keywords_with_weights = list(zip(top_keywords, weights))

    # 종료시간 확인
    end_time = time.time()

    # 소요시간 출력
    print(f"{len(series_word_list)}개 댓글 키워드 추출에 걸린 시간: {end_time - start_time} 초\n")

    return top_keywords, keywords_with_weights

In [33]:
from rank_bm25 import BM25Okapi
import time
import numpy as np
import pandas as pd
from collections import defaultdict

# BM25 기반 키워드 및 상대적 비중 추출 함수 (min_df 추가)
def extract_keywords_bm25(series_word_list, top_n=100, min_df=1):
    # 시작시간 확인
    start_time = time.time()

    # 시리즈의 각 리스트를 문서로 간주하여 BM25 모델 생성
    tokenized_corpus = series_word_list.tolist()
    bm25 = BM25Okapi(tokenized_corpus)

    # 각 단어의 출현 빈도를 저장할 딕셔너리
    doc_frequency = defaultdict(int)

    # 문서 내 각 단어의 출현 빈도 계산
    for doc in tokenized_corpus:
        unique_words_in_doc = set(doc)
        for word in unique_words_in_doc:
            doc_frequency[word] += 1

    # 각 단어의 BM25 점수를 저장할 딕셔너리
    word_scores = defaultdict(float)
    total_docs = len(tokenized_corpus)

    # 각 문서에서의 BM25 점수를 계산하여 해당 단어에 대한 점수를 합산
    for doc in tokenized_corpus:
        scores = bm25.get_scores(doc)
        for word, score in zip(doc, scores):
            # 단어의 출현 빈도가 min_df 이상인 경우에만 점수 계산
            if doc_frequency[word] >= min_df:
                word_scores[word] += score

    # 문서 전체에 대한 평균 BM25 점수로 변환
    word_scores = {word: score / total_docs for word, score in word_scores.items()}

    # 상위 BM25 점수를 기준으로 키워드 추출
    sorted_words = sorted(word_scores.items(), key=lambda item: item[1], reverse=True)
    top_keywords = sorted_words[:top_n]

    # 가중치를 전체 합으로 표준화
    total_weight = sum(score for _, score in top_keywords)
    keywords_with_weights = [(word, score / total_weight) for word, score in top_keywords]

    # 종료시간 확인
    end_time = time.time()

    # 소요시간 출력
    print(f"{len(series_word_list)}개 댓글 키워드 추출에 걸린 시간: {end_time - start_time} 초\n")

    return top_keywords, keywords_with_weights

# 각 레벨 별로 키워드 추출

In [41]:
## keyword_list 초기화 및 설정
keyword_list = pd.DataFrame(columns=['Group', 'Title', 'keyword'])

## 모든 댓글에 대한 키워드 추가
#sample_df['word_list'] = sample_df['word_list'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# 단순 빈도 기반 키워드 추출
keywords, weights = extract_keywords_frequency(sample_df['word_list'], top_n=30, min_df=5)
keyword_list['keyword'] = keywords
keyword_list['weights'] = weights
keyword_list['Group'] = 'All'
keyword_list['Title'] = 'All'
## 각 그룹에 대한 추가키워드 발견
# Group 열에 속한 그룹들의 리스트
groups = sample_df['Group'].unique()

# 기존 keyword_list에서 추출된 키워드와 중복되지 않도록 새로운 키워드 추출
# existing_keywords = keyword_list['keyword'].tolist()

# 그룹별로 새로운 키워드 추출
for group in groups:
    word_list = sample_df[sample_df['Group'] == group]['word_list']
    keywords, weights = extract_keywords_frequency(word_list, top_n=30, min_df=5)
    
    # 새로운 키워드를 기존 리스트에 추가
    frac = pd.DataFrame()
    frac['keyword'] = keywords
    frac['weights'] = weights
    frac['Group'] = group
    frac['Title'] = 'All'
    
    keyword_list = pd.concat([keyword_list, frac])
    keyword_list.drop_duplicates(subset='keyword', keep='first')

## 각 뮤비에 대한 키워드 추출
# Title 열에 속한 그룹들의 리스트
titles = sample_df['Title'].unique()

# MV별로 새로운 키워드 추출
for title in titles:
    group = sample_df[sample_df['Title'] == title]['Group'].iloc[0] # 해당 타이틀에 속한 그룹 조회
    word_list = sample_df[sample_df['Title'] == title]['word_list'] # 댓글 시리즈 가져오기
    keywords, weights = extract_keywords_frequency(word_list, top_n=30, min_df=3)
    
    # 새로운 키워드를 기존 리스트에 추가
    frac = pd.DataFrame()
    frac['keyword'] = keywords
    frac['weights'] = weights
    frac['Group'] = group
    frac['Title'] = title
    
    keyword_list = pd.concat([keyword_list, frac])
    keyword_list.drop_duplicates(subset='keyword', keep='first')

# 인덱스 초기화
keyword_list = keyword_list.reset_index(drop=True)

648060개 댓글 키워드 추출에 걸린 시간: 0.9479749202728271 초

23511개 댓글 키워드 추출에 걸린 시간: 0.02622079849243164 초

50699개 댓글 키워드 추출에 걸린 시간: 0.056355953216552734 초

249967개 댓글 키워드 추출에 걸린 시간: 0.4786069393157959 초

45376개 댓글 키워드 추출에 걸린 시간: 0.05791211128234863 초

7425개 댓글 키워드 추출에 걸린 시간: 0.0075719356536865234 초

136993개 댓글 키워드 추출에 걸린 시간: 0.13891816139221191 초

33181개 댓글 키워드 추출에 걸린 시간: 0.03650403022766113 초

7846개 댓글 키워드 추출에 걸린 시간: 0.00917196273803711 초

69125개 댓글 키워드 추출에 걸린 시간: 0.07907819747924805 초

23937개 댓글 키워드 추출에 걸린 시간: 0.023941993713378906 초

4339개 댓글 키워드 추출에 걸린 시간: 0.005378007888793945 초

2622개 댓글 키워드 추출에 걸린 시간: 0.0028009414672851562 초

551개 댓글 키워드 추출에 걸린 시간: 0.0005419254302978516 초

1442개 댓글 키워드 추출에 걸린 시간: 0.0016677379608154297 초

1406개 댓글 키워드 추출에 걸린 시간: 0.0013339519500732422 초

1416개 댓글 키워드 추출에 걸린 시간: 0.0014011859893798828 초

1916개 댓글 키워드 추출에 걸린 시간: 0.0019330978393554688 초

1608개 댓글 키워드 추출에 걸린 시간: 0.0015621185302734375 초

570개 댓글 키워드 추출에 걸린 시간: 0.0005812644958496094 초

3550개 댓글 키워드 추출에 걸린 시간: 0.00438

In [42]:
# 출력 옵션 설정: 모든 열을 출력
pd.set_option('display.max_columns', None)  # 모든 열 출력
pd.set_option('display.max_rows', None)     # 모든 행 출력

#keyword_list

In [48]:
# 전체 키워드에서 드랍할 키워드 목록
keywords_to_drop = ['뉴진스', '아이브', '하이브', '민희진', '단월드', '에스파', '사이비', '르세라핌', '방시혁', '리즈', '혜인', '하니', '다니엘', '아일릿', 
                    '여자친구', '민지', '레이', '엔믹스', '유진', '원영', '가을', '피프티', '박지원', '스테이씨', '버니즈', '어도어', '이브', '카리나', 
                    '마고', '이서', '윈터', '정신병원', '해린', '류진', '마그네틱', '장원영', '다이브', '단군신화', '침착맨', '디토', '방탄소년단', '오엠지', 
                    '안유진', '지젤', '닝닝', '혜진', '예지', '리아', '수가']


# 'Group'이 'All'이고 'Keyword'가 keywords_to_drop에 포함된 행 드랍
keyword_list = keyword_list[~((keyword_list['Group'] == 'All') & (keyword_list['keyword'].isin(keywords_to_drop)))]
keyword_list.to_csv('keyword_list.csv')

In [49]:
# 'Group'과 'Title'로 그룹화한 후, Keyword를 와이드 포맷으로 변환
keyword_list['Keyword_Index'] = keyword_list.groupby(['Group', 'Title']).cumcount() + 1

# 'Keyword_Index'를 열로 피봇하여 와이드 포맷으로 변환
df_wide_format = keyword_list.pivot(index=['Group', 'Title'], columns='Keyword_Index', values='keyword')

# 결과 출력
df_wide_format.to_csv('keyword_list_wide_format.csv')
df_wide_format

Keyword_Index                                                1     2      3   \
Group       Title                                                              
All         All                                              노래    뮤비     사랑   
FIFTY FIFTY All                                              노래   피프티     대표   
            Cupid                                            노래    대표    피프티   
            Higher                                           노래    음색    피프티   
            Lovin' Me                                        노래   피프티     뮤비   
            SOS                                              노래   피프티     응원   
            Starry Night                                     노래   피프티     응원   
ITZY        All                                              노래    사랑     류진   
            BET ON ME                                        노래    사랑     류진   
            BORN TO BE                                       노래    사랑     리아   
            Boys Like You                                    노래    사랑     리아   
            CAKE                                             노래    사랑    케이크   
            Cheshire                                         노래    사랑     체셔   
            ICY                                              노래   한국인     사랑   
            LOCO                                             노래    사랑     예지   
            Mr. Vampire                                      노래    리아     사랑   
            None of My Business                              노래    사랑     류진   
            Not Shy                                          노래    사랑     류진   
            SNEAKERS                                         노래    사랑   스니커즈   
            SWIPE                                            노래   타이틀     뮤비   
            UNTOUCHABLE                                      노래    사랑     컴백   
            Voltage                                          노래    사랑     일본   
            WANNABE                                          노래    류진     사랑   
            달라달라(DALLA DALLA)                          노래    채령    한국인   
            마.피.아. In the morning                         노래   마피아     가사   
IVE         Accendio                                       아센디오   아이브     뮤비   
            After LIKE                                      아이브    노래     사랑   
            All                                             아이브    노래     사랑   
            All Night (Feat. Saweetie)                      아이브    노래     사랑   
            Baddie                                          아이브    노래     사랑   
            ELEVEN                                          아이브    노래     사랑   
            Either Way                                      아이브    노래     사랑   
            I AM                                            아이브    노래     사랑   
            Kitsch                                           리즈    노래    아이브   
            LOVE DIVE                                       아이브    노래    다이브   
            Off The Record                                  아이브    노래     사랑   
            해야 (HEYA)                                      중국    한국    아이브   
Kep1er      All                                              노래   케플러     사랑   
            Giddy                                            노래   케플러     사랑   
            Grand Prix                                      케플러    사랑     노래   
            I do! Do you                                     노래   케플러     한국   
            Straight Line                                   케플러    노래     사랑   
            Sugar Rush                                      케플러    노래    NaN   
            Up!                                              노래   케플러     사랑   
            WA DA DA                                         노래   케플러     사랑   
            We Fresh                                        케플러    노래     사랑   
            Wing Wing                                        노래   케플러     일본   
LE SSERAFIM ANTIFRAGILE                                      노래  르세라핌     중독   
